### data filtering

#### imports

In [39]:
import pandas as pd
import re
import os
import pandasql as ps

#### functions

In [40]:
def get_next_filename(base_filename, folder):
    """
    Генерує унікальну назву файлу, додаючи +1 до номера.
    """
    if not os.path.exists(folder):
        os.makedirs(folder)

    files = os.listdir(folder)

    matching_files = [f for f in files if f.startswith(base_filename) and f.endswith(".csv")]

    max_number = 0
    for file in matching_files:
        try:
            number = int(file.replace(base_filename, "").replace(".csv", "").strip("_"))
            if number > max_number:
                max_number = number
        except ValueError:
            continue

    next_number = max_number + 1
    return os.path.join(folder, f"{base_filename}_{next_number}.csv")
def process_dataset(df):
    """
    Processes the dataset according to the specified requirements:
    1. Filters rows where position contains 'Data Scientist' or 'Data Science'.
    2. Sorts rows by the 'period' field.
    3. Adds a new column 'final_seniority' with the first non-empty value from seniority -> local experience seniority -> general experience seniority.
    4. Adds a new column 'final_salary' as a concatenation of salary, bonuses flag, and bonuses amount.
    5. Repeats rows based on the 'frequency' column value (converted to int). Defaults to 1 if frequency is missing or NaN.
    6. Keeps only the columns: 'period', 'final_seniority', 'final_salary'.

    :param df: Input DataFrame.
    :return: Processed DataFrame.
    """
    # Step 1: Filter rows where position contains 'Data Scientist' or 'Data Science'
    filtered_df = df[df['position'].str.contains('Data Scientist|Data Science', na=False)]

    # Step 2: Sort rows by the 'period' field
    sorted_df = filtered_df.sort_values(by='period')

    # Списки категорій
    junior_values = [
        '1', '1.5', '2', '0.25', 'менше 3 місяців', '0.5', '1 рік', '2 роки', 'півроку', '3 місяці', 
        'менше 3 місяців', '2.0', '1.0', '0.0', 'менше як 3 місяці', 'пів року', '1,5 року', '1-3 роки', 
        'до року', 'junior', 'немає тайтлу', 'intern/trainee', "меньше 3 месяцев"
    ]
    middle_values = [
        '3', '4', '5', '4 роки', '3 роки', '5 років', '5.0', '4.0', '3.0', '4-6 років', 'middle'
    ]
    senior_values = [
        '10 и более лет', '8', '9', '6', '7', '10 і більше років', '7 років', '6 років', '8 років', 'senior', '7.0', '8.0', '9.0', '10.0', '9 років',
        '7-10 років', '10+ років', 'Tech Lead', 'Senior', 'Team Lead', 'Manager', 'Architect','Lead/Team Lead', 'Principal', '6.0', 'head'
    ]
    
    # Функція для визначення категорії
    def map_seniority(value):
        if pd.isna(value):  # Якщо значення NaN, повертаємо None
            return None
        value = str(value).strip().lower()  # Приводимо значення до нижнього регістру для порівняння
        if value in [v.lower() for v in junior_values]:
            return 'junior'
        elif value in [v.lower() for v in middle_values]:
            return 'middle'
        elif value in [v.lower() for v in senior_values]:
            return 'senior'
        return value  # Якщо значення не знайдено в жодній категорії
    
    # Step 1: Додати колонку 'final_seniority'
    sorted_df['final_seniority'] = sorted_df.apply(
        lambda row: next(
            (val for val in [row['seniority'], row['general experience seniority'], row['local experience seniority']] if pd.notna(val)),
            None
        ),
        axis=1
    )
    
    # Step 2: Оновити значення в колонці 'final_seniority' залежно від категорій
    sorted_df['final_seniority'] = sorted_df['final_seniority'].apply(map_seniority)
    # 
    # Step 4: Add 'final_salary' column
    sorted_df['final_salary'] = sorted_df.apply(
        lambda row: f"{row['salary']} {row['bonuses flag']} {row['bonuses amount']}",
        axis=1
    )
    processed_df = apply_frequency(sorted_df)
    final_columns = ['period', 'position', "final_seniority", "final_salary"]
    combined_df = processed_df[final_columns]

    return combined_df


def apply_frequency(df):
    # Заповнюємо пропущені значення у 'frequency' значенням 1.0
    df['frequency'] = df['frequency'].fillna(1.0)

    # Замінюємо від'ємні значення на 0
    df['frequency'] = df['frequency'].apply(lambda x: max(x, 0))

    # Перетворюємо значення у 'frequency' на цілі числа
    df['frequency'] = df['frequency'].astype(int)

    # Повторюємо рядки на основі значення у frequency
    df = df.loc[df.index.repeat(df['frequency'])].reset_index(drop=True)
    return df

#### variables

In [41]:
input_path = "raw_salary_compile/combined_salary_2.csv"
output_folder = "filtered_dataset" 

#### main functions

In [42]:
# Usage Example
# Assuming `df` is your input DataFrame

output_path = get_next_filename("filtered_dataset", output_folder)
df = pd.read_csv(input_path, low_memory=False)  # Load your dataset

processed_df = process_dataset(df)  # Process the dataset
processed_df.to_csv(output_path, index=False)

In [43]:
data = pd.read_csv("filtered_dataset/filtered_dataset_3.csv", low_memory=False)
query = """
    SELECT DISTINCT `final_seniority`
    FROM data
"""

# Виконання запиту
unique_bonuses_flag = ps.sqldf(query, locals())

# Виведення результату
print(unique_bonuses_flag)

  final_seniority
0          junior
1          senior
2          middle


In [44]:
data1 = pd.read_csv("filtered_dataset/filtered_dataset_3.csv", low_memory=False)
query = """
    SELECT count(*), final_seniority FROM data 
    group by final_seniority
"""

# Виконання запиту
unique_bonuses_flag = ps.sqldf(query, locals())

# Виведення результату
print(unique_bonuses_flag)

   count(*) final_seniority
0      1459          junior
1       962          middle
2       717          senior
